<a href="https://colab.research.google.com/github/aswit3/Start_Your_NLP_Career/blob/master/spam_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

In [27]:
!unzip drive/My\ Drive/datasets/sms-spam-collection-dataset.zip

Archive:  drive/My Drive/datasets/sms-spam-collection-dataset.zip
replace spam.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: spam.csv                


In [0]:
option = 2

if option == 1:
  url = "https://tfhub.dev/google/elmo/2"
  embed = hub.Module(url)
elif option == 2:
  url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
  embed = hub.Module(url)

In [29]:
data = pd.read_csv('spam.csv', encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [30]:
y = list(data['v1'])
x = list(data['v2'])

le = preprocessing.LabelEncoder()
le.fit(y)

LabelEncoder()

In [0]:
def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le, ['ham', 'spam', 'ham', 'ham'])

untest = decode(le, test)


In [0]:
x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:5000])
y_train = np.asarray(y_enc[:5000])

x_test = np.asarray(x_enc[5000:])
y_test = np.asarray(y_enc[5000:])

In [33]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

input_text = Input(shape=(1,), dtype=tf.string)

if option == 1:
  def ELMoEmbedding(x):
      return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]
  embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
elif option == 2:
  def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))
  embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)

  
#embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=32)
    model.save_weights('./model.h5')

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model.h5')  
    predicts = model.predict(x_test, batch_size=32)

y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

Epoch 1/1
5000/5000 [==============================] - 127s 25ms/step - loss: 0.1451 - acc: 0.9486
[[491   7]
 [  5  69]]
              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       498
        spam       0.91      0.93      0.92        74

    accuracy                           0.98       572
   macro avg       0.95      0.96      0.95       572
weighted avg       0.98      0.98      0.98       572

